### Convert PDF downloaded from internet to a tabular format for processing

In [56]:
import pdfplumber
import pandas as pd

# Path to your PDF file
pdf_file = "HS CODE catalogue.pdf"

# Open the PDF file
with pdfplumber.open(pdf_file) as pdf:
    all_data = []
    
    # Iterate through all pages
    for page_num, page in enumerate(pdf.pages):
        # Extract tables from the page. REALLY IMPORTANT TO SET THE X TOLERANCE
        tables = page.extract_tables(table_settings={"text_x_tolerance":1})
        
        # If tables are found, process them
        if tables:
            for table in tables:
                all_data.extend(table)

# Convert the extracted data into a pandas DataFrame
df = pd.DataFrame(all_data)

# Convertir primer renglon en header
df.columns = df.iloc[0]  
df = df.drop(df.index[0]) 
df = df.reset_index(drop=True)

# Reemplazar caracteres conflictivos y eliminar columnas inecesarias
df['Product Description'] = df['Product Description'].str.replace('-', '*')

# Reemplazar saltos de linea que'\n' se tomaron erroneamente como string durante el proceso de PDF-CSV
df['Product Description'] = df['Product Description'].str.replace('\n', ' ')

## Regresar a '-' solo cuando se usa como separador entre la misma palabra (Pure-bred)
#def replace_asterisk_between_letters(text):
#    return re.sub(r'(?<=[a-zA-Z])\*(?=[a-zA-Z])', '-', text)
#
## Limpiar
#df['Product Description'] = df['Product Description'].apply(replace_asterisk_between_letters)

df.head(15)

# Save the DataFrame to an Excel file
df.to_csv("HS code Catalogue_AsTable.csv", encoding='utf-8', index=False)

### Format the new Excel sheet (remove unwanted columns, blank rows, etc)


In [57]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('HS code Catalogue_AsTable.csv')

#Mantener solo las primeras 2 columnas que tienen info relevante
df = df.drop(df.columns[2:28], axis=1)

## Convertir primer renglon en header
#df.columns = df.iloc[0]  
#df = df.drop(df.index[0]) 
#df = df.reset_index(drop=True)

#Remover renglones vacios completos
df = df.dropna(how='all')

# Quitar los renglones donde se repiten encabezados
df_clean = df[df['Product Description'] != 'Product Description']

# Replace the newline character '\n' with a space in the 'Product Description' column
#df_clean['Product Description'] = df_clean['Product Description'].str.replace('\n', ' ')
df_clean.head(40)


,HS Code,Product Description
1,01,Chapter 1 * Live animals
3,01.01,"Live horses, asses, mules and hinnies."
4,NaN,* Horses:
5,0101.21.00,* * Pure*bred breeding animals
6,0101.29.00,* * Other
7,0101.30,* Asses:
8,0101.30.10,* * Pure*bred breeding animals
9,0101.30.90,* * Other
10,0101.90.00,* Other
12,01.02,Live bovine animals.


### Concatenar la descripcion de la parte, con la subparte para mejor contexto

In [81]:
# Function to concatenate descriptions based on the number of asterisks
def concatenate_descriptions(df):
    last_upper_word = ""
    last_star_word = ""
    last_double_star_word = ""
    last_triple_star_word = ""
    
    result = []
    
    for index, row in df.iterrows():
        desc = row['Product Description'].strip()

        if isinstance(desc, (int, float)):  #Added only for quality check
            print(f"{desc} is a number.")            
        #if desc.startswith('* * * * '):
        #    # Quadruple asterisk, concatenate with the last triple-star word
        #    result.append(last_upper_word + " ----> " + desc[4:].strip())
        #elif desc.startswith('* * * '):
        #    # Triple asterisk, concatenate with the last double-star word
        #    result.append(last_upper_word + " ---> " + desc[3:].strip())
        #    last_triple_star_word = last_double_star_word + " " + desc[3:].strip()
        if desc.startswith('* * ') or desc.startswith('* * * ') or desc.startswith('* * * * '):
            # Double asterisk, concatenate with the last upper word
            result.append(last_upper_word + last_star_word + " --> " + desc[2:].strip())
            last_double_star_word = " - - >" + desc[2:].strip()
        elif desc.startswith('* '):
            # Double asterisk, concatenate with the last upper word
            result.append(last_upper_word + " -> " + desc[1:].strip())
            last_star_word = " - > " + desc[1:].strip()            
        else:
            # Normal word, update the last upper word and store as is
            last_upper_word = desc
            result.append(desc)
    
    df['Full description'] = result
    return df

# Apply the function to the DataFrame
concatenated_df = concatenate_descriptions(df_clean)

# Reemplazar caracteres conflictivos y eliminar columnas inecesarias
#concatenated_df['Full description'] = concatenated_df['Full description'].str.replace('-', '*')
concatenated_df = concatenated_df.drop(columns=["Product Description"])

concatenated_df['Full description'] = concatenated_df['Full description'].str.replace('*', ' ')
concatenated_df['Full description'] = concatenated_df['Full description'].str.replace("’", "'")

#Eliminar renglones sin codigo
concatenated_df = concatenated_df.dropna()

concatenated_df.head(55)

# Guardar el archivo Excel que fungira como HS Catalogue para entrenar el LLM
concatenated_df.to_csv("HS code Catalogue_Processed.csv",encoding='utf-8', index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_3724\2655951521.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Full description'] = result
